<a href="https://colab.research.google.com/github/aravind2060/Dijkstra_and_kruskal/blob/master/Shortest_path_Algorithm_Dijkstra_and_Kruskal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import heapq
import time

In [51]:
class DijkstraAlgo:
    def __init__(self, num_vertices, directed=False):
        # Step 1: Initialize the Dijkstra's Algorithm class
        self.num_vertices = num_vertices
        self.graph = {i: [] for i in range(num_vertices)}  # Adjacency list representation
        self.directed = directed
        self.vertex_map = {}  # Map to store vertex labels to integer mapping
        self.reverse_map = {}  # Reverse mapping from integers to labels

    def add_vertex(self, vertex):
        # Step 2: Add a vertex to the graph
        if vertex not in self.vertex_map:
            index = len(self.vertex_map)
            self.vertex_map[vertex] = index
            self.reverse_map[index] = vertex

    def add_edge(self, u, v, weight):
        # Step 3: Add an edge between two vertices with a weight
        if u not in self.vertex_map:
            self.add_vertex(u)
        if v not in self.vertex_map:
            self.add_vertex(v)
        u_index = self.vertex_map[u]
        v_index = self.vertex_map[v]
        self.graph[u_index].append((v_index, weight))  # Directed edge from u to v
        if not self.directed:
            self.graph[v_index].append((u_index, weight))  # Add reverse edge for undirected graph

    def print_graph(self):
        # Step 4: Print the graph structure
        for vertex in self.graph:
            print(f"{self.reverse_map[vertex]} ->", end=" ")
            for edge in self.graph[vertex]:
                print(f"({self.reverse_map[edge[0]]}, {edge[1]})", end=" ")
            print()

    def dijkstra_logic(self, source):
        # Step 5: Implement Dijkstra's Algorithm to find shortest paths from source
        distances = {vertex: float('infinity') for vertex in self.graph}
        distances[source] = 0
        priority_queue = [(0, source)]  # Initialize priority queue with (distance, vertex)
        predecessors = {vertex: None for vertex in self.graph}

        while priority_queue:
            current_distance, current_vertex = heapq.heappop(priority_queue)
            if current_distance > distances[current_vertex]:
                continue  # Skip processing if current path is not the shortest

            for neighbor, weight in self.graph[current_vertex]:
                distance = current_distance + weight
                if distance < distances[neighbor]:
                    distances[neighbor] = distance  # Update shortest distance to neighbor
                    predecessors[neighbor] = current_vertex  # Track predecessor for path reconstruction
                    heapq.heappush(priority_queue, (distance, neighbor))  # Add neighbor to queue

        return distances, predecessors

    def get_path(self, source, destination, predecessors):
        # Step 6: Retrieve the shortest path from source to destination using predecessors
        path = []
        step = destination
        while step is not None:
            path.append(step)
            step = predecessors[step]
        path.reverse()  # Reverse to get path from source to destination
        return [self.reverse_map[node] for node in path if node in self.reverse_map]

In [52]:
class KruskalAlgo:
    def __init__(self, vertices):
        # Step 1: Initialize the Kruskal's Algorithm class
        self.V = vertices
        self.graph = []  # List to store edges in the format (weight, u, v)

    def add_edge(self, u, v, w):
        # Step 2: Add an edge to the graph with its weight
        self.graph.append((w, u, v))  # Store edge as a tuple (weight, u, v)

    # Iterative find with path compression
    def find(self, parent, i):
        # Step 3: Find the root of the set containing vertex i with path compression
        if parent[i] != i:
            parent[i] = self.find(parent, parent[i])  # Path compression
        return parent[i]

    # Union by rank
    def union(self, parent, rank, x, y):
        # Step 4: Union two sets using union by rank
        xroot = self.find(parent, x)
        yroot = self.find(parent, y)

        if xroot != yroot:
            if rank[xroot] < rank[yroot]:
                parent[xroot] = yroot
            elif rank[xroot] > rank[yroot]:
                parent[yroot] = xroot
            else:
                parent[yroot] = xroot
                rank[xroot] += 1
            return True  # Return True if union is successful (no cycle)
        return False  # Return False if x and y are already in the same set

    def kruskal_logic(self):
        # Step 5: Implement Kruskal's Algorithm to find Minimum Spanning Tree (MST)
        result = []                   # List to store edges of the MST
        self.graph.sort()             # Sort edges by their weight
        parent = list(range(self.V))  # Initialize parent array for Union-Find
        rank = [0] * self.V           # Initialize rank array for Union-Find
        e = 0                         # Counter for edges in MST

        for weight, u, v in self.graph:
            if e == self.V - 1:
                break                 # Stop if MST is complete (V-1 edges)
            if self.union(parent, rank, u, v):
                result.append((u, v, weight))  # Add edge to MST
                e += 1

        return result                 # Return the edges of the MST

In [53]:
def load_graph_from_file_for_dijkstra(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        first_line = lines[0].split()
        num_vertices = int(first_line[0])
        graph_type = first_line[2]
        directed = graph_type == 'D'

        g = DijkstraAlgo(num_vertices, directed)
        vertex_map = {}
        vertex_index = 0

        # Read each line from the second line to the second last line
        for line in lines[1:-1]:
            parts = line.strip().split()
            if len(parts) == 3:
                u, v, w = parts[0], parts[1], int(parts[2])

                # Ensure each vertex label is mapped to a unique integer
                if u not in vertex_map:
                    vertex_map[u] = vertex_index
                    g.add_vertex(u)
                    vertex_index += 1
                if v not in vertex_map:
                    vertex_map[v] = vertex_index
                    g.add_vertex(v)
                    vertex_index += 1

                # Add edge to the graph using mapped vertex indices
                g.add_edge(u, v, w)

        # Handle optional source node (last line in the file)
        source = lines[-1].strip()
        if source in vertex_map:
            source = vertex_map[source]
        else:
            source = None

        return g, source

In [54]:
def load_graph_from_file_for_kruskal(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        first_line = lines[0].split()
        num_vertices = int(first_line[0])

        g = KruskalAlgo(num_vertices)  # Initialize KruskalAlgo instance
        vertex_map = {}  # Dictionary to map vertex labels to integers
        vertex_index = 0  # Index counter for assigning unique integers to vertices

        for line in lines[1:]:
            parts = line.strip().split()
            if len(parts) == 3:
                u, v, w = parts[0], parts[1], int(parts[2])

                # Ensure each vertex label is mapped to a unique integer
                if u not in vertex_map:
                    vertex_map[u] = vertex_index
                    vertex_index += 1
                if v not in vertex_map:
                    vertex_map[v] = vertex_index
                    vertex_index += 1

                g.add_edge(vertex_map[u], vertex_map[v], w)  # Add edge to KruskalAlgo instance

        return g, vertex_map

In [55]:
# List of file paths
file_paths = [
              '/content/sample_data/input1.txt',
              '/content/sample_data/input2.txt',
              '/content/sample_data/input3.txt',
              '/content/sample_data/input4.txt'
              ]

# Iterate over each file path
for file_path in file_paths:
    print(f"Processing file: {file_path}")

    # Load graph from file
    graph, source = load_graph_from_file_for_dijkstra(file_path)

    # Print graph structure
    print("Input Graph structure:")
    graph.print_graph()

    # Print source vertex if specified
    if source is not None:
        print(f"Source vertex: {graph.reverse_map[source]}")
    else:
        print("No source vertex specified.")

    # Start measuring the time
    start_time = time.time()

    # Run Dijkstra's algorithm
    distances, predecessors = graph.dijkstra_logic(source)

    # Calculate the time taken
    end_time = time.time()
    time_taken = end_time - start_time
    print("Time taken for Dijkstra:", time_taken, "seconds\n")

    # Print paths and distances from source
    print("Paths and Distances from Source:")
    for vertex in graph.graph:
        path = graph.get_path(source, vertex, predecessors)
        print(f"Path from {graph.reverse_map[source]} to {graph.reverse_map[vertex]}: {' -> '.join(path)} with cost {distances[vertex]}")

    print()  # Print empty line for better readability between different files

Processing file: /content/sample_data/input1.txt
Input Graph structure:
A -> (B, 1) (C, 2) 
B -> (A, 1) (C, 1) (D, 3) (E, 2) 
C -> (A, 2) (B, 1) (D, 1) (E, 2) 
D -> (B, 3) (C, 1) (E, 4) (F, 3) 
E -> (B, 2) (C, 2) (D, 4) (F, 3) 
F -> (D, 3) (E, 3) 
Source vertex: A
Time taken for Dijkstra: 3.743171691894531e-05 seconds

Paths and Distances from Source:
Path from A to A: A with cost 0
Path from A to B: A -> B with cost 1
Path from A to C: A -> C with cost 2
Path from A to D: A -> C -> D with cost 3
Path from A to E: A -> B -> E with cost 3
Path from A to F: A -> C -> D -> F with cost 6

Processing file: /content/sample_data/input2.txt
Input Graph structure:
A -> (B, 2) (C, 3) 
B -> (C, 1) (D, 4) 
C -> (D, 2) (E, 1) 
D -> (A, 1) (F, 2) 
E -> (D, 3) (F, 2) 
F -> (G, 3) 
G -> (E, 1) (A, 2) 
Source vertex: A
Time taken for Dijkstra: 3.504753112792969e-05 seconds

Paths and Distances from Source:
Path from A to A: A with cost 0
Path from A to B: A -> B with cost 2
Path from A to C: A -> C wit

In [56]:
# List of file paths
file_paths = ['/content/sample_data/input1.txt',
              '/content/sample_data/input2.txt',
              '/content/sample_data/input3.txt',
              '/content/sample_data/input4.txt']

# Iterate over each file path
for file_path in file_paths:
    print(f"Processing file: {file_path}")

    # Load graph from file
    graph, vertex_map = load_graph_from_file_for_kruskal(file_path)

    # Print to verify mapping
    print("Vertex Map:", vertex_map)

    # Start measuring the time
    start_time = time.time()

    # Compute the Minimum Spanning Tree using Kruskal's Algorithm
    mst = graph.kruskal_logic()

    # Calculate the time taken
    end_time = time.time()
    time_taken = end_time - start_time
    print("Time taken for Kruskal's Algorithm:", time_taken, "seconds\n")

    # Calculate and print the total cost of the Minimum Spanning Tree
    total_cost = sum(weight for weight, _, _ in mst)
    print("Total cost of the Minimum Spanning Tree:", total_cost)

    # Print the edges in the Minimum Spanning Tree
    print("Edges in the Minimum Spanning Tree:")
    for weight, u, v in mst:
        # Mapping integer indexes back to original labels for printing
        u_label = list(vertex_map.keys())[list(vertex_map.values()).index(u)]
        v_label = list(vertex_map.keys())[list(vertex_map.values()).index(v)]
        print(f"({u_label}, {v_label}) - {weight}")

    print()  # Print empty line for better readability between different files

Processing file: /content/sample_data/input1.txt
Vertex Map: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5}
Time taken for Kruskal's Algorithm: 4.315376281738281e-05 seconds

Total cost of the Minimum Spanning Tree: 7
Edges in the Minimum Spanning Tree:
(B, B) - 0
(C, B) - 1
(D, B) - 2
(E, C) - 1
(F, D) - 3

Processing file: /content/sample_data/input2.txt
Vertex Map: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6}
Time taken for Kruskal's Algorithm: 2.6702880859375e-05 seconds

Total cost of the Minimum Spanning Tree: 15
Edges in the Minimum Spanning Tree:
(C, B) - 1
(E, B) - 2
(A, B) - 3
(E, B) - 6
(B, C) - 0
(F, C) - 3

Processing file: /content/sample_data/input3.txt
Vertex Map: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7}
Time taken for Kruskal's Algorithm: 2.9325485229492188e-05 seconds

Total cost of the Minimum Spanning Tree: 16
Edges in the Minimum Spanning Tree:
(D, B) - 0
(C, B) - 1
(F, B) - 4
(B, C) - 0
(G, C) - 3
(H, C) - 6
(F, D) - 2

Proces